# Decoding Strategies in LLMs

### Pytorch Requirement

In [54]:
import torch
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
from tqdm.notebook import tqdm
import math

### Load GPT2 Model

In [55]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_path = "/home/ub/WD8T/cbh/gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
model.eval()

text = "I have a dream"
input_ids = tokenizer.encode(text, return_tensors='pt').to(device)

outputs = model.generate(input_ids, max_length=len(input_ids.squeeze())+5, pad_token_id=50256)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated text: {generated_text}")

Generated text: I have a dream of being a doctor.


GPT-2, a autoregressive models, predict the next token in a sequence based on the preceding tokens. Consider a sequence of token $w = (w_1, w_2, ..., w_t)$. The joint probability of this sequence $P(w)$ can be broken down as:

$$
P(w) = \prod \limits^{n}_{i= 1} P(w_i | w_1 ... w_{i - 1})
$$

For each token $w_i$ in the sequence, $P(w_i | w_1 ... w_{i - 1})$ represents the conditional probability of $w_i$ given all the preceding tokens $w_1 ... w_n$. GPT-2 calculates this conditional probability for each of the tokens in its vocabulary.

This leads to the question: how do we use these probabilities to generate next token? There are several decoding strategies, such as greedy search, beam search, Top-K sampling and Nucleus sampling, that come into play.


In [56]:
def get_log_prob(logits, token_id):
    # Compute the softmax of the logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    log_probabilities = torch.log(probabilities)
    # Get the log probability of the token
    token_log_probability = log_probabilities[token_id].item()

    return token_log_probability

### Greedy Search

Greedy search is a decoding method that takes the most probable token at each step as the next token in the sequence. To put it simply, it only retains the most likely token at each stage, discarding all other potential options.

While this approach might sound intuitive, it's important to note that the greedy search is short-sighted, because it only considers the most probable token at each step without considering the overall effort on the sequence. This property makes it fast and efficient as it does not need to keep track of multiple sequences, but it also means that it can miss out on better sequences that might have appeared with slightly less probable next tokens.

Now we implement the strategies of Greedy Search.

In [57]:
def greedy_search(model, input_ids):
    outputs = model(input_ids)
    predictions = outputs.logits  # shape = ([1, 4, 50257])

    # Using the last token outputs to predict next sub-word
    logits = predictions[0, -1, :]
    
    token_id = torch.argmax(logits).unsqueeze(0)

    # Compute the score of the predicted token
    log_token_score = get_log_prob(logits, token_id)

    token_score = math.exp(log_token_score)

    new_input_ids = torch.cat([input_ids, token_id.unsqueeze(0)], dim=-1)

    return new_input_ids, token_score

new_input_ids = input_ids
num_of_generate_token = 5

for i in range(1, num_of_generate_token + 1):
    new_input_ids, token_score = greedy_search(model, new_input_ids)
    print(f"Stage {i}: {tokenizer.decode(new_input_ids[0])}")
    print(f"New token score:{tokenizer.decode(new_input_ids[0, -1])} -> {token_score * 100:.2f}%")

Stage 1: I have a dream of
New token score: of -> 15.20%
Stage 2: I have a dream of being
New token score: being -> 9.68%
Stage 3: I have a dream of being a
New token score: a -> 30.59%
Stage 4: I have a dream of being a doctor
New token score: doctor -> 2.86%
Stage 5: I have a dream of being a doctor.
New token score:. -> 22.93%


Well, you can see the output of our implement of greedy search is similar to the original output of GPT-2.

### Beam Search

Unlike greedy search, which only consider the next most probable token, beam search takes into account the $n$ most likely tokens, where $n$ presents the number of beams. This procedure is repeated until a predefine maximum length is reached or an end-of-sequence token appears. At this point, the sequence (or "beam") with the highest overall score is chosen as the output.

Let's first take a look at the output of the original GPT-2 model when using beam search to generate text.

In [58]:
# Using the generate function and setting the parameter of num_beams as 5
outputs = model.generate(input_ids, max_length=len(input_ids.squeeze())+5, num_beams = 4, pad_token_id = 50256)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated text: {generated_text}")

Generated text: I have a dream. I have a dream


Alright, you can see the model generated the sentence 'I have a dream. I want to be.' Now, let's implement our own beam search.

In [59]:
def greedy_sampling(logits, beams):
    k = torch.topk(logits, beams)
    
    return k.indices

def beam_search(model, inputs_ids, max_length = 5, num_beams = 2, sampling = greedy_sampling):
    beams = [(inputs_ids, 0)]  # (sequence, score)

    for _ in range(max_length):
        # store the next beams
        paticipate = []
        for sequence, score in beams:
            output = model(sequence)
            prediction = output.logits

            logits = prediction[0, -1, :]

            top_k_token = sampling(logits, num_beams)

            for next_token in top_k_token.unsqueeze(1):
                new_input_ids = torch.cat([sequence, next_token.unsqueeze(0)], dim = -1)
                new_token_score = get_log_prob(logits, next_token)
                paticipate.append((new_input_ids, score + new_token_score))

        beams = paticipate

    sorted_beams = sorted(beams, key = lambda x: x[1], reverse = True)

    return sorted_beams

new_input_ids = input_ids
# Parameters
num_of_generate_token = 5
beams = 2

best_sequence = beam_search(model, new_input_ids, num_of_generate_token, beams)

for i in range(3):
    generated_text = tokenizer.decode(best_sequence[i][0].squeeze(0), skip_special_tokens=True)
    print(f"The {i + 1} place of generation text: {generated_text}\nThe cumulative score of the sentence: {best_sequence[i][1]:.3f}")

The 1 place of generation text: I have a dream. I have a dream
The cumulative score of the sentence: -6.252
The 2 place of generation text: I have a dream. I want to be
The cumulative score of the sentence: -6.674
The 3 place of generation text: I have a dream. I want to go
The cumulative score of the sentence: -7.928


### Top-k sampling

Top-k sampling is a technique that leverages the probability distribution generated by the language model to select **a token randomly from the K most likely options**.

Another way of introducing randomness is the concept of temperature. The temperature $T$ is a parameter that ranges from $0$ to $1$, which affects the probabilities generated by the softmax function, making the most likely tokens more influential. In pratice, it simply consists of dividing the input logits by a value we call temperature:

$$
\text{softmax}(x_i) = \frac{e^{x_i / T}}{\sum_j e^{x_j/T}}
$$

Well, let's now implement the top k sampling algorithm. 

In [60]:
def top_k_sampling(logits, beams, temperature = 1, top_k = 20):
    indices_to_remove = logits < torch.topk(logits, top_k)[0][-1]
    new_logits = torch.clone(logits)
    new_logits[indices_to_remove] = float("-inf")

    prob = torch.nn.functional.softmax(new_logits / temperature, dim = -1)

    next_tokens = torch.multinomial(prob, beams)

    return next_tokens

new_input_ids = input_ids
# Parameters
num_of_generate_token = 5
beams = 2

best_sequence = beam_search(model, new_input_ids, num_of_generate_token, beams, top_k_sampling)

for i in range(3):
    generated_text = tokenizer.decode(best_sequence[i][0].squeeze(0), skip_special_tokens=True)
    print(f"The {i + 1} place of generation text: {generated_text}\nThe cumulative score of the sentence: {best_sequence[i][1]:.3f}")

The 1 place of generation text: I have a dream job and I want to
The cumulative score of the sentence: -9.102
The 2 place of generation text: I have a dream job.

"
The cumulative score of the sentence: -10.815
The 3 place of generation text: I have a dream for the future."

The cumulative score of the sentence: -10.928


### Nucleus sampling

Nucleus sampling, also known as top-p sampling, taks a different approach from top-k sampling. Rather then selecting the $k$ most probable tokens, nucleus sampling chooses a cutoff value $p$ such that the **sum of the probabilities of the selected tokens exceeds $p$**. This forms a "nucleus" of tokens from which to randomly choose the next token.

This variability often results in a more diverse and creative output, making nucleus sampling popular for tasks such as text generation.

Now we implement our nucleus sampling function.

In [61]:
def nucleus_sampling(logits, beams, temperature = 1.0, p = 0.7):
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    prob = torch.nn.functional.softmax(sorted_logits / temperature, dim = -1)
    cum_prob = torch.cumsum(prob, dim = -1)

    mask = cum_prob < p

    if mask.sum() > beams:
        top_p_index_to_keep = torch.where(mask)[0][-1].detach().cpu().tolist()
    else:
        top_p_index_to_keep = beams

    indices_to_remove = sorted_indices[top_p_index_to_keep:]
    sorted_logits[indices_to_remove] = float('-inf')

    probabilities = torch.nn.functional.softmax(sorted_logits / temperature, dim=-1)
    next_tokens = torch.multinomial(probabilities, beams)

    return next_tokens

new_input_ids = input_ids
# Parameters
num_of_generate_token = 12
beams = 2

best_sequence = beam_search(model, new_input_ids, num_of_generate_token, beams, nucleus_sampling)

for i in range(3):
    generated_text = tokenizer.decode(best_sequence[i][0].squeeze(0), skip_special_tokens=True)
    print(f"The {i + 1} place of generation text: {generated_text}\nThe cumulative score of the sentence: {best_sequence[i][1]:.3f}")

The 1 place of generation text: I have a dream. That is my dream. It is my dream. I
The cumulative score of the sentence: -20.510
The 2 place of generation text: I have a dream. That is my dream. It is my dream.

The cumulative score of the sentence: -21.389
The 3 place of generation text: I have a dream. That is my dream.

"I don't
The cumulative score of the sentence: -21.672


### Conclusion

In this article, we have delved deep into various decoding methods used by LLMs, specifically GPT-2. Understanding these techniques and their trade-offs will equip you to better guide the LLMs towards producing increasingly realistic, nuanced, and compelling textual output.

### Reference

* [Decoding Strategies in Large Language Models](https://mlabonne.github.io/blog/posts/2023-06-07-Decoding_strategies.html)
* [Beam Search](https://medium.com/@jingying.h95/beam-search-1e875fd59ab4)